In [186]:
import numpy as np
import matplotlib.pyplot as plt

In [187]:
states = np.array([[0,0,0,0],   # 0
                   [0,0,0,1],   # 1
                   [0,0,1,0],   # 2
                   [0,0,1,1],   # 3
                   [0,1,0,0],   # 4
                   [0,1,0,1],   # 5
                   [0,1,1,0],   # 6
                   [0,1,1,1],   # 7
                   [1,0,0,0],   # 8
                   [1,0,0,1],   # 9
                   [1,0,1,0],   # 10
                   [1,0,1,1],   # 11
                   [1,1,0,0],   # 12
                   [1,1,0,1],   # 13
                   [1,1,1,0],   # 14
                   [1,1,1,1]])  # 15

# Create additional states
new_states = np.vstack((states, np.array([[0,0,0,0]])))

num_states = states.shape[0]

actions = np.array([[0,0,0,0],  # 0
                    [1,0,0,0],  # 1
                    [0,1,0,0],  # 2
                    [0,0,1,0],  # 3
                    [0,0,0,1]]) # 4

num_actions = actions.shape[0]

connectivity_matrix = \
    np.array([[0,0,-1,0],
              [1,0,-1,-1],
              [0,1,0,0],
              [-1,1,1,0],])

def normalize(v):
    v[v < 0] = 0
    v[v > 0] = 1

    return v

def getNextState(state, action, noise):
    cm_s = np.dot(connectivity_matrix, state)
    cm_s = normalize(cm_s)

    next_state = np.logical_xor(cm_s, action)
    next_state = np.logical_xor(next_state, noise)

    return next_state.astype(int)

# new_states = getNextState(states[7], actions[4], np.array([0,0,0,0]))
# print(new_states)

# Calculate the reward for a given state and action
def getReward(state_index,action_index):
    return np.sum(states[state_index]*5)-np.linalg.norm(actions[action_index], ord=1)

    # if action_index == 1 or action_index == 2:
    #     return -1
    # else:
    #     return np.sum(states[state_index]*5)-np.linalg.norm(actions[action_index], ord=1)



# Calculate the noise for a given probability
def getNoise(prob):
    return np.random.binomial(n=1,p=prob,size=4)

def getTransitionProbabilities(prob):
    p = np.zeros((num_actions,num_states,num_states))
    r = np.zeros((num_actions,num_states,num_states))
    
    for a in range(num_actions):
        for i in range(num_states):
            for j in range(num_states):
                power_var = np.linalg.norm(states[j]-np.logical_xor(normalize(np.dot(connectivity_matrix, states[i])), actions[a]),ord=1)
                
                p[a][i][j] = (prob**power_var) * ((1-prob)**(4-power_var))
                r[a][i][j] = getReward(j,a)
    return p,r



        

def getRSA(p_matrix, r_matrix):
    rsa = np.zeros((num_actions, num_states))

    for a in range(num_actions):
        for s in range(num_states):
            for s_prime in range(num_states):
                rsa[a][s] += (p_matrix[a][s][s_prime] * r_matrix[a][s][s_prime])
    
    return rsa

### Part a

In [188]:
prob = 0.05
theta = 0.01
gamma = 0.95

def value_iteration(p, rsa, gamma, theta):
    v_0 = np.zeros((num_actions, num_states))
    v_1 = np.zeros((num_actions, num_states))
    v_result = np.zeros((num_states))

    while True:
        delta = 0
        v_0 = np.copy(v_1)
        v_1 = np.zeros((num_actions, num_states))

        for a in range(num_actions):
            v_1[a] = rsa[a] + gamma * np.matmul(p[a], v_0[a].reshape(num_states,1)).reshape(num_states)

        for s in range(num_states):
            # Find the maximum value state for all actions
            v_result[s] = np.max(v_1[:,s])
            
            # Always set the value of the goal state to 0
            v_result[15] = 0

            for i in range(num_actions):
                delta = max(delta, abs(v_0[i][s] - v_1[i][s]))
            
            if(delta < theta):
                return v_result,v_0,delta
            


p,r = getTransitionProbabilities(prob)

rsa = getRSA(p,r)

v_result, q, delta = value_iteration(p, rsa, gamma, theta)

optimal_policy = []

for i in range(num_states):
    optimal_policy.append(np.argmax(q[:,i]))

print(optimal_policy)




[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [200]:
from random import randrange

num_episodes = 100
num_steps = 200


def getAverageActivation(random = False):
    AvgA = 0
    for e in range(num_episodes):
        sum_s = 0

        # Select random start state
        s = randrange(num_states)
        
        # print(f'Start State: {s}')

        for i in range(num_steps):
            # print(f'State: {s}')

            # Get the next state
            if random:
                next_state = getNextState(states[s], actions[optimal_policy[s]], getNoise(prob))
            else:
                next_state = getNextState(states[s], actions[0], getNoise(prob))

            
            # Get the index of the next state
            s = np.where(np.all(states == next_state, axis=1))[0][0]
            # print(f'Next State: {next_state}')

            # Calculate the sum of the states
            sum_s += np.linalg.norm(states[next_state], ord=1)
            # print(f'Sum: {sum_s}')

        # print(sum_s/200)
        AvgA += sum_s/num_steps
        # print(AvgA)
    
    return (AvgA/num_episodes)
    
print(getAverageActivation(True))
print(getAverageActivation(False))



2.84155
0.4727999999999998


### Part b

$p = 0.2$, $\gamma=0.95$, $\theta=0.01$

In [202]:
prob = 0.2

p,r = getTransitionProbabilities(prob)

rsa = getRSA(p,r)

v_result, q, delta = value_iteration(p, rsa, gamma, theta)

optimal_policy = []
for i in range(num_states):
    optimal_policy.append(np.argmax(q[:,i]))

print(optimal_policy)
print(getAverageActivation(True))
print(getAverageActivation(False))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
2.3572500000000005
1.2585500000000003


$p = 0.45$, $\gamma=0.95$, $\theta=0.01$

In [203]:
prob = 0.45

p,r = getTransitionProbabilities(prob)

rsa = getRSA(p,r)

v_result, q, delta = value_iteration(p, rsa, gamma, theta)

optimal_policy = []
for i in range(num_states):
    optimal_policy.append(np.argmax(q[:,i]))

print(optimal_policy)
print(getAverageActivation(True))
print(getAverageActivation(False))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1.9201000000000008
1.9107499999999993


### Part c